In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!nvidia-smi

Wed Dec 28 06:59:41 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P0    26W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install tensorflow-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 588.3 MB 21 kB/s 
     |████████████████████████████████| 439 kB 68.8 MB/s 
     |████████████████████████████████| 6.0 MB 66.3 MB/s 
     |████████████████████████████████| 1.7 MB 69.8 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.9.1
    Uninstalling tensorboard-2.9.1:
      Successfully uninstalled tensorboard-2.9.1
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Successfully uninstalled keras-2.9.0
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.12
    Uninstalling flatbuffers-1.12:
      Successfully u

In [ ]:
train_path = "/content/drive/MyDrive/Data"

In [ ]:
import numpy as np
import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os
from tensorflow.python.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [ ]:
img_height, img_width = 254, 254
batch_size = 32
train_data = tf.keras.preprocessing.image_dataset_from_directory(
           train_path,
           subset="training",
           validation_split=0.3,
           seed=123,
           label_mode = 'categorical',
           image_size = (img_height, img_width),
            batch_size = batch_size
)


Found 1016 files belonging to 2 classes.
Using 712 files for training.


In [ ]:
img_height, img_width = 254, 254
batch_size = 32
test_data = tf.keras.preprocessing.image_dataset_from_directory(
           train_path,
           subset="validation",
           validation_split=0.3,
           seed=123,
           label_mode = 'categorical',
           image_size = (img_height, img_width),
            batch_size = batch_size
)

Found 1016 files belonging to 2 classes.
Using 304 files for validation.


In [ ]:
class_name = train_data.class_names
print(class_name)

['Covid', 'Normal']


In [ ]:
from keras.engine.sequential import Sequential
resnet_model = Sequential()
default_model = tf.keras.applications.ResNet50(
          include_top=False,
          weights='imagenet',
         input_shape=(254,254, 3),
         pooling='avg',
         classes=2)
       
for layer in default_model.layers:
          layer.trainable = False
        
resnet_model.add(default_model)
resnet_model.add(Flatten())
resnet_model.add(Dense(512, activation='relu'))
resnet_model.add(Dense(2, activation='sigmoid'))

94765736/94765736 [==============================] - 5s 0us/step


In [ ]:
resnet_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 module_wrapper (ModuleWrapp  (None, 2048)             0         
 er)                                                             
                                                                 
 module_wrapper_1 (ModuleWra  (None, 512)              1049088   
 pper)                                                           
                                                                 
 module_wrapper_2 (ModuleWra  (None, 2)                1026      
 pper)                                                           
                                                                 
Total params: 24,637,826
Trainable params: 1,050,114
Non-trainable params: 23,587,712
____________________________________

In [ ]:
resnet_model.compile(optimizer=Adam(learning_rate=0.001), loss = 'binary_crossentropy',metrics=['accuracy'])

In [ ]:
resnet_model.fit(
           train_data,
           validation_data = test_data,
           epochs= 10
)

Epoch 1/10
23/23 [==============================] - 367s 12s/step - loss: 0.7399 - accuracy: 0.7065 - val_loss: 0.3547 - val_accuracy: 0.8421
Epoch 2/10
23/23 [==============================] - 5s 204ms/step - loss: 0.2865 - accuracy: 0.8722 - val_loss: 0.2624 - val_accuracy: 0.8750
Epoch 3/10
23/23 [==============================] - 5s 203ms/step - loss: 0.2078 - accuracy: 0.9270 - val_loss: 0.2095 - val_accuracy: 0.9178
Epoch 4/10
23/23 [==============================] - 5s 204ms/step - loss: 0.1550 - accuracy: 0.9551 - val_loss: 0.1907 - val_accuracy: 0.9112
Epoch 5/10
23/23 [==============================] - 5s 204ms/step - loss: 0.1254 - accuracy: 0.9649 - val_loss: 0.1772 - val_accuracy: 0.9276
Epoch 6/10
23/23 [==============================] - 5s 204ms/step - loss: 0.0988 - accuracy: 0.9691 - val_loss: 0.1755 - val_accuracy: 0.9243
Epoch 7/10
23/23 [==============================] - 5s 204ms/step - loss: 0.0824 - accuracy: 0.9817 - val_loss: 0.1424 - val_accuracy: 0.9474
Epoch 